In [2]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

In [69]:
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()
df = df[(df.league=="Barclays Premier League") & (df.season >= 2019)]

In [70]:
df['h_margin'] = df['score1'] - df['score2']

In [71]:
results = []

In [72]:
for x in df['h_margin']:
  if x > 0:
    results.append("HOME WIN")
  elif x == 0:
    results.append("DRAW")
  else:
    results.append("AWAY WIN")

In [73]:
df['result'] = results

In [74]:
df['h_prob_margin'] = df['prob1'] - df['prob2']
prob_margins = []

In [75]:
for y in df['h_prob_margin']:
  if y >= 0:
    prob_margins.append("HOME FAVORITE")
  else:
    prob_margins.append("AWAY FAVORITE")

In [76]:
df['prob_bucket'] = prob_margins

In [77]:
df = df.drop(columns=['h_margin', 'h_prob_margin'])

In [78]:
comparison_df = df[['result','prob_bucket']]
comparisons = []

In [79]:
ran = range(0, len(comparison_df), 1)

In [80]:
for z in ran:
  z = comparison_df.iloc[z,:]
  if (z[0]=="HOME WIN") & (z[1]=="HOME FAVORITE"):
    comparisons.append("EXPECTED HOME")
  elif (z[0]=="AWAY WIN") & (z[1]=="AWAY FAVORITE"):
    comparisons.append("EXPECTED AWAY")
  elif (z[0]=="AWAY WIN") & (z[1]=="HOME FAVORITE"):
    comparisons.append("UNEXPECTED AWAY")
  elif (z[0]=="HOME WIN") & (z[1]=="AWAY FAVORITE"):
    comparisons.append("UNEXPECTED HOME")
  else:
    comparisons.append("NO DECISIVE OUTCOME")

In [81]:
df['result_type'] = comparisons

In [87]:
X = df[['score1','score2','prob1','prob2']]
y = df['result_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [88]:
model = XGBClassifier(objective='multiclass:softprob')

In [89]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [90]:
predictions = model.predict(X_test)

In [92]:
accuracy = accuracy_score(y_test, predictions)

In [94]:
print("Test Accuracy: {}%".format(accuracy * 100))

Test Accuracy: 98.68421052631578%
